<a href="https://colab.research.google.com/github/pbajpai21/MLOps-Jan2025/blob/main/MLOps3_G24AIT008.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Roll No : G24AIT008

# Q1. (Dataset and Model Preparation)

Load the Dataset

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

# Define the transformation: Convert images to tensors and normalize
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load Fashion-MNIST dataset
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)


100%|██████████| 26.4M/26.4M [00:01<00:00, 15.6MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 333kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 5.55MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 7.30MB/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



Train-Validation Split (80:20)

In [2]:
# Define train-validation split (80:20)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_data, val_data = random_split(train_dataset, [train_size, val_size])

# Create DataLoader for training, validation, and test sets
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Defining Model Architecture

In [3]:
import torch.nn as nn

class FashionMNISTModel(nn.Module):
    def __init__(self, hidden_units):
        super(FashionMNISTModel, self).__init__()
        self.fc1 = nn.Linear(28 * 28, hidden_units)  # First layer
        self.relu = nn.ReLU()  # Activation function
        self.fc2 = nn.Linear(hidden_units, 10)  # Output layer

    def forward(self, x):
        x = x.view(x.shape[0], -1)  # Flatten input
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Set default hidden layer neurons
hidden_units = 256
model = FashionMNISTModel(hidden_units)


# Q2. (Setting Up the Project & Logging Hyperparameters)

Initialize wandb project

In [4]:
!pip install wandb
import wandb

wandb.login(relogin=True)
wandb.init(project="MLOps2025_g24ait008", config={
    "learning_rate": 0.001,
    "batch_size": 64,
    "epochs": 5,
    "hidden_units": hidden_units
})


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: g24ait008 (g24ait008-ml-ops) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Defining Loss function and Optimizer

In [5]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)


# Q3. (Training and Validation)

Training the model

In [6]:
import time

def train(model, train_loader, val_loader, criterion, optimizer, epochs):
    start_time = time.time()
    model.train()

    for epoch in range(epochs):
        train_loss, train_correct = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_correct += (outputs.argmax(dim=1) == labels).sum().item()

        train_loss /= len(train_loader)
        train_acc = train_correct / len(train_loader.dataset)

        # Validation
        val_loss, val_correct = 0, 0
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_correct += (outputs.argmax(dim=1) == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = val_correct / len(val_loader.dataset)

        # Log metrics in wandb
        wandb.log({"Train Loss": train_loss, "Train Accuracy": train_acc,
                   "Validation Loss": val_loss, "Validation Accuracy": val_acc})

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")
    end_time = time.time()
    execution_time = round(end_time - start_time, 2)
    wandb.log({"Time spent (seconds)" : execution_time})
    print(f"Time spent: {execution_time} seconds")

try:
  train(model, train_loader, val_loader, criterion, optimizer, wandb.config.epochs)
finally:
  # Close wandb session
  wandb.finish()


Epoch [1/5], Train Loss: 0.5118, Train Accuracy: 0.8149, Validation Loss: 0.4388, Validation Accuracy: 0.8414
Epoch [2/5], Train Loss: 0.3889, Train Accuracy: 0.8587, Validation Loss: 0.3574, Validation Accuracy: 0.8710
Epoch [3/5], Train Loss: 0.3459, Train Accuracy: 0.8721, Validation Loss: 0.3331, Validation Accuracy: 0.8776
Epoch [4/5], Train Loss: 0.3183, Train Accuracy: 0.8843, Validation Loss: 0.3391, Validation Accuracy: 0.8721
Epoch [5/5], Train Loss: 0.2985, Train Accuracy: 0.8910, Validation Loss: 0.3166, Validation Accuracy: 0.8823
Time spent: 115.16 seconds


Time spent (seconds),▁
Train Accuracy,▁▅▆▇█
Train Loss,█▄▃▂▁
Validation Accuracy,▁▆▇▆█
Validation Loss,█▃▂▂▁
Time spent (seconds),115.16
Train Accuracy,0.891
Train Loss,0.29854
Validation Accuracy,0.88233
Validation Loss,0.31659


# Q4. Hyperparameter Exploration

Defining Sweep configuration

In [7]:
sweep_config = {
    "method": "random",
    "parameters": {
        "hidden_units": {
            "values": [128, 256, 512]  # Different values for the hidden layer
        },
        "batch_size": {"values": [128, 256]},
        "epochs": {"value": 5},
        "learning_rate": {
            "values": [0.002, 0.01, 0.1]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="MLOps2025_g24ait008")


Create sweep with ID: l5dm6euw
Sweep URL: https://wandb.ai/g24ait008-ml-ops/MLOps2025_g24ait008/sweeps/l5dm6euw


Running the sweep

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def sweep_train(config=None):
    wandb.init(config=config)
    model = FashionMNISTModel(wandb.config.hidden_units).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train(model, train_loader, val_loader, criterion, optimizer, epochs=5)


wandb.agent(sweep_id, sweep_train, count=10)


wandb: Agent Starting Run: 10xsqrhq with config:
wandb: 	batch_size: 256
wandb: 	epochs: 5
wandb: 	hidden_units: 256
wandb: 	learning_rate: 0.002


Epoch [1/5], Train Loss: 0.5094, Train Accuracy: 0.8150, Validation Loss: 0.4252, Validation Accuracy: 0.8398
Epoch [2/5], Train Loss: 0.3862, Train Accuracy: 0.8582, Validation Loss: 0.3816, Validation Accuracy: 0.8580
Epoch [3/5], Train Loss: 0.3449, Train Accuracy: 0.8748, Validation Loss: 0.3484, Validation Accuracy: 0.8698
Epoch [4/5], Train Loss: 0.3208, Train Accuracy: 0.8817, Validation Loss: 0.3379, Validation Accuracy: 0.8734
Epoch [5/5], Train Loss: 0.3029, Train Accuracy: 0.8882, Validation Loss: 0.3136, Validation Accuracy: 0.8867
Time spent: 106.1 seconds


Time spent (seconds),▁
Train Accuracy,▁▅▇▇█
Train Loss,█▄▂▂▁
Validation Accuracy,▁▄▅▆█
Validation Loss,█▅▃▃▁
Time spent (seconds),106.1
Train Accuracy,0.88825
Train Loss,0.30292
Validation Accuracy,0.88667
Validation Loss,0.31359


wandb: Agent Starting Run: e52bjqce with config:
wandb: 	batch_size: 256
wandb: 	epochs: 5
wandb: 	hidden_units: 128
wandb: 	learning_rate: 0.1


Epoch [1/5], Train Loss: 0.5257, Train Accuracy: 0.8136, Validation Loss: 0.4631, Validation Accuracy: 0.8290
Epoch [2/5], Train Loss: 0.3978, Train Accuracy: 0.8551, Validation Loss: 0.3575, Validation Accuracy: 0.8703
Epoch [3/5], Train Loss: 0.3580, Train Accuracy: 0.8700, Validation Loss: 0.3619, Validation Accuracy: 0.8662
Epoch [4/5], Train Loss: 0.3305, Train Accuracy: 0.8797, Validation Loss: 0.3515, Validation Accuracy: 0.8715
Epoch [5/5], Train Loss: 0.3122, Train Accuracy: 0.8849, Validation Loss: 0.3234, Validation Accuracy: 0.8780
Time spent: 87.9 seconds


...

Epoch [1/5], Train Loss: 0.5071, Train Accuracy: 0.8160, Validation Loss: 0.4554, Validation Accuracy: 0.8267
Epoch [2/5], Train Loss: 0.3854, Train Accuracy: 0.8587, Validation Loss: 0.3640, Validation Accuracy: 0.8678
Epoch [3/5], Train Loss: 0.3457, Train Accuracy: 0.8719, Validation Loss: 0.3485, Validation Accuracy: 0.8666
Epoch [4/5], Train Loss: 0.3180, Train Accuracy: 0.8827, Validation Loss: 0.3216, Validation Accuracy: 0.8801
Epoch [5/5], Train Loss: 0.2954, Train Accuracy: 0.8908, Validation Loss: 0.3315, Validation Accuracy: 0.8832
Time spent: 92.83 seconds


Time spent (seconds),▁
Train Accuracy,▁▅▆▇█
Train Loss,█▄▃▂▁
Validation Accuracy,▁▆▆██
Validation Loss,█▃▂▁▂
Time spent (seconds),92.83
Train Accuracy,0.89079
Train Loss,0.29539
Validation Accuracy,0.88325
Validation Loss,0.33154


wandb: Agent Starting Run: 5w9z4jkc with config:
wandb: 	batch_size: 128
wandb: 	epochs: 5
wandb: 	hidden_units: 256
wandb: 	learning_rate: 0.1


Epoch [1/5], Train Loss: 0.5039, Train Accuracy: 0.8175, Validation Loss: 0.4254, Validation Accuracy: 0.8429
Epoch [2/5], Train Loss: 0.3826, Train Accuracy: 0.8599, Validation Loss: 0.3524, Validation Accuracy: 0.8697
Epoch [3/5], Train Loss: 0.3448, Train Accuracy: 0.8731, Validation Loss: 0.3495, Validation Accuracy: 0.8707
Epoch [4/5], Train Loss: 0.3187, Train Accuracy: 0.8824, Validation Loss: 0.3201, Validation Accuracy: 0.8810
Epoch [5/5], Train Loss: 0.2987, Train Accuracy: 0.8900, Validation Loss: 0.3170, Validation Accuracy: 0.8855
Time spent: 93.42 seconds


Time spent (seconds),▁
Train Accuracy,▁▅▆▇█
Train Loss,█▄▃▂▁
Validation Accuracy,▁▅▆▇█
Validation Loss,█▃▃▁▁
Time spent (seconds),93.42
Train Accuracy,0.89002
Train Loss,0.29872
Validation Accuracy,0.8855
Validation Loss,0.31696


# Q5. (Artifact Management and Model Saving)

Saving the model

In [9]:
torch.save(model.state_dict(), "MLOps2025.pth")
wandb.init()
artifact = wandb.Artifact("MLOps2025", type="model")
artifact.add_file("MLOps2025.pth")
wandb.log_artifact(artifact)

wandb.finish()